In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import pymysql

In [2]:
pollution_file = "pollution_us_2000_2016.csv"
pollution_df = pd.read_csv(pollution_file)
pollution_df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [3]:
list(pollution_df.columns.values)

['Unnamed: 0',
 'State Code',
 'County Code',
 'Site Num',
 'Address',
 'State',
 'County',
 'City',
 'Date Local',
 'NO2 Units',
 'NO2 Mean',
 'NO2 1st Max Value',
 'NO2 1st Max Hour',
 'NO2 AQI',
 'O3 Units',
 'O3 Mean',
 'O3 1st Max Value',
 'O3 1st Max Hour',
 'O3 AQI',
 'SO2 Units',
 'SO2 Mean',
 'SO2 1st Max Value',
 'SO2 1st Max Hour',
 'SO2 AQI',
 'CO Units',
 'CO Mean',
 'CO 1st Max Value',
 'CO 1st Max Hour',
 'CO AQI']

In [4]:
poll_df = pollution_df[['State', 'County', 'Date Local', 'NO2 Mean', 'O3 Mean', 'SO2 Mean', 'CO Mean']].copy()
poll_df.head()

,State,County,Date Local,NO2 Mean,O3 Mean,SO2 Mean,CO Mean
0,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,1.145833
1,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,0.878947
2,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,1.145833
3,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,0.878947
4,Arizona,Maricopa,2000-01-02,22.958333,0.013375,1.958333,0.850000


In [5]:
poll_df.columns = ['State', 'County', 'Date', 'NO2 Mean (PPB)', 'O3 Mean (PPM)', 'SO2 Mean (PPB)', 'CO Mean (PPM)']
poll_df.head()

,State,County,Date,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
0,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,1.145833
1,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,0.878947
2,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,1.145833
3,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,0.878947
4,Arizona,Maricopa,2000-01-02,22.958333,0.013375,1.958333,0.850000


In [6]:
poll_df['Date'] = pd.to_datetime(poll_df['Date'])

In [7]:
poll_df['Date'].dt.year

0          2000
1          2000
2          2000
3          2000
4          2000
5          2000
6          2000
7          2000
8          2000
9          2000
10         2000
11         2000
12         2000
13         2000
14         2000
15         2000
16         2000
17         2000
18         2000
19         2000
20         2000
21         2000
22         2000
23         2000
24         2000
25         2000
26         2000
27         2000
28         2000
29         2000
           ... 
1746631    2016
1746632    2016
1746633    2016
1746634    2016
1746635    2016
1746636    2016
1746637    2016
1746638    2016
1746639    2016
1746640    2016
1746641    2016
1746642    2016
1746643    2016
1746644    2016
1746645    2016
1746646    2016
1746647    2016
1746648    2016
1746649    2016
1746650    2016
1746651    2016
1746652    2016
1746653    2016
1746654    2016
1746655    2016
1746656    2016
1746657    2016
1746658    2016
1746659    2016
1746660    2016
Name: Date, Length: 1746

In [8]:
poll_df['Year'] = poll_df['Date'].dt.year
poll_df.head()

,State,County,Date,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM),Year
0,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,1.145833,2000
1,Arizona,Maricopa,2000-01-01,19.041667,0.022500,3.000000,0.878947,2000
2,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,1.145833,2000
3,Arizona,Maricopa,2000-01-01,19.041667,0.022500,2.975000,0.878947,2000
4,Arizona,Maricopa,2000-01-02,22.958333,0.013375,1.958333,0.850000,2000


In [9]:
poll_df = poll_df[['State', 'County', 'Date', 'Year', 'NO2 Mean (PPB)', 'O3 Mean (PPM)', 'SO2 Mean (PPB)', 'CO Mean (PPM)']]
poll_df.head()

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
0,Arizona,Maricopa,2000-01-01,2000,19.041667,0.022500,3.000000,1.145833
1,Arizona,Maricopa,2000-01-01,2000,19.041667,0.022500,3.000000,0.878947
2,Arizona,Maricopa,2000-01-01,2000,19.041667,0.022500,2.975000,1.145833
3,Arizona,Maricopa,2000-01-01,2000,19.041667,0.022500,2.975000,0.878947
4,Arizona,Maricopa,2000-01-02,2000,22.958333,0.013375,1.958333,0.850000


In [10]:
poll_2010_df = poll_df.loc[(poll_df['Year'].isin(['2010'])) & (poll_df['State']!= 'Country Of Mexico')]
poll_2010_df.head()

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
968407,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.333333,1.037500
968408,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.333333,0.973684
968409,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.287500,1.037500
968410,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.287500,0.973684
968411,Arizona,Maricopa,2010-01-02,2010,35.500000,0.010000,4.083333,0.987500


In [11]:
bad_NO2_2010 = (poll_2010_df[poll_2010_df["NO2 Mean (PPB)"] > 53])
bad_NO2_2010

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
989099,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.675000,1.734783
989100,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.675000,1.687500
989101,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.662500,1.734783
989102,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.662500,1.687500
993207,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.663636,1.381818
993208,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.663636,1.387500
993209,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.716667,1.381818
993210,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.716667,1.387500
993219,California,Los Angeles,2010-01-11,2010,54.317391,0.001375,2.860870,1.326087
993220,California,Los Angeles,2010-01-11,2010,54.317391,0.001375,2.860870,1.425000


In [12]:
bad_NO2_2010 = (poll_2010_df[poll_2010_df["NO2 Mean (PPB)"] > 53].groupby(["State"]).count())
bad_NO2_2010

,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
State,,,,,,,
California,16,16,16,16,16,16,16
Pennsylvania,12,12,12,12,12,12,12


In [13]:
bad_O3_2010 = (poll_2010_df[poll_2010_df["O3 Mean (PPM)"] > 0.07])
bad_O3_2010

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
1006039,California,Santa Barbara,2010-09-26,2010,2.217391,0.070417,2.086957,0.339130
1006040,California,Santa Barbara,2010-09-26,2010,2.217391,0.070417,2.086957,0.345833
1006041,California,Santa Barbara,2010-09-26,2010,2.217391,0.070417,2.057143,0.339130
1006042,California,Santa Barbara,2010-09-26,2010,2.217391,0.070417,2.057143,0.345833
1045534,Ohio,Medina,2010-05-05,2010,7.047619,0.070778,0.142857,0.209524
1045535,Ohio,Medina,2010-05-05,2010,7.047619,0.070778,0.142857,0.200000
1045536,Ohio,Medina,2010-05-05,2010,7.047619,0.070778,0.128571,0.209524
1045537,Ohio,Medina,2010-05-05,2010,7.047619,0.070778,0.128571,0.200000
1045618,Ohio,Medina,2010-05-27,2010,4.125000,0.070708,9.083333,0.204167
1045619,Ohio,Medina,2010-05-27,2010,4.125000,0.070708,9.083333,0.200000


In [14]:
bad_O3_2010 = (poll_2010_df[poll_2010_df["O3 Mean (PPM)"] > 0.07].groupby(["State"]).count())
bad_O3_2010

,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
State,,,,,,,
California,4,4,4,4,4,4,4
Ohio,8,8,8,8,8,8,8
Tennessee,24,24,24,24,24,24,24
Virginia,4,4,4,4,4,4,4


In [15]:
poll_2010_df = poll_df.loc[(poll_df['Year'].isin(['2010'])) & (poll_df['State']!= 'Country Of Mexico')]
poll_2010_df.head()

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
968407,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.333333,1.037500
968408,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.333333,0.973684
968409,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.287500,1.037500
968410,Arizona,Maricopa,2010-01-01,2010,32.208333,0.013208,3.287500,0.973684
968411,Arizona,Maricopa,2010-01-02,2010,35.500000,0.010000,4.083333,0.987500


In [16]:
bad_NO2_2010 = (poll_2010_df[poll_2010_df["NO2 Mean (PPB)"] > 53])
bad_NO2_2010

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
989099,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.675000,1.734783
989100,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.675000,1.687500
989101,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.662500,1.734783
989102,California,Los Angeles,2010-01-07,2010,56.913043,0.004750,2.662500,1.687500
993207,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.663636,1.381818
993208,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.663636,1.387500
993209,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.716667,1.381818
993210,California,Los Angeles,2010-01-08,2010,53.622727,0.002667,1.716667,1.387500
993219,California,Los Angeles,2010-01-11,2010,54.317391,0.001375,2.860870,1.326087
993220,California,Los Angeles,2010-01-11,2010,54.317391,0.001375,2.860870,1.425000


In [17]:
bad_NO2_2010 = (poll_2010_df.loc[poll_2010_df["NO2 Mean (PPB)"] > 53, ["State", "NO2 Mean (PPB)"]]).groupby(["State"]).count()
bad_NO2_2010

,NO2 Mean (PPB)
State,
California,16
Pennsylvania,12


In [18]:
total_baddays_2010 = bad_O3_2010.merge(bad_NO2_2010, on="State", how="outer")
total_baddays_2010[["O3 Mean (PPM)","NO2 Mean (PPB)_y"]]

,O3 Mean (PPM),NO2 Mean (PPB)_y
State,,
California,4.0,16.0
Ohio,8.0,NaN
Tennessee,24.0,NaN
Virginia,4.0,NaN
Pennsylvania,NaN,12.0


In [19]:
#total_baddays_2010 = total_baddays_2010.rename(columns={"O3 Mean (PPM)":"O3 Mean (PPM) 2010", "NO2 Mean (PPB)_y":"NO2 Mean (PPB) 2010":})
#total_baddays_2010.head()

In [20]:
pollution_data_2010 = total_baddays_2010.rename(columns={"O3 Mean (PPM)":"O3 Mean (PPM) 2010", "NO2 Mean (PPB)":"NO2 Mean (PPB) 2010"})
pollution_data_2010

,County,Date,Year,NO2 Mean (PPB)_x,O3 Mean (PPM) 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y
State,,,,,,,,
California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0
Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN
Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN
Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN
Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0


In [21]:
poll_2014_df = poll_df.loc[poll_df['Year'].isin(['2014'])]
poll_2014_df.head()

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
1464412,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.363636,0.204545
1464413,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.363636,0.200000
1464414,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.242857,0.204545
1464415,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.242857,0.200000
1464416,Alabama,Jefferson,2014-01-02,2014,3.6625,0.018458,-0.009091,0.158824


In [22]:
bad_O3_2014 = (poll_2014_df[poll_2014_df["O3 Mean (PPM)"] > 0.07])
bad_O3_2014

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
1499830,California,San Bernardino,2014-05-17,2014,6.518182,0.074708,1.277273,0.431818
1499831,California,San Bernardino,2014-05-17,2014,6.518182,0.074708,1.277273,0.445833
1499832,California,San Bernardino,2014-05-17,2014,6.518182,0.074708,1.228571,0.431818
1499833,California,San Bernardino,2014-05-17,2014,6.518182,0.074708,1.228571,0.445833
1499910,California,San Bernardino,2014-06-06,2014,10.295455,0.070167,1.418182,0.295455
1499911,California,San Bernardino,2014-06-06,2014,10.295455,0.070167,1.418182,0.295833
1499912,California,San Bernardino,2014-06-06,2014,10.295455,0.070167,1.385714,0.295455
1499913,California,San Bernardino,2014-06-06,2014,10.295455,0.070167,1.385714,0.295833
1504050,California,Santa Barbara,2014-10-05,2014,0.956522,0.070850,0.347826,0.000000
1504051,California,Santa Barbara,2014-10-05,2014,0.956522,0.070850,0.347826,0.000000


In [23]:
bad_O3_2014 = (poll_2014_df.loc[poll_2014_df["O3 Mean (PPM)"] > 0.07, ["State", "O3 Mean (PPM)"]]).groupby(["State"]).count()
bad_O3_2014

,O3 Mean (PPM)
State,
California,12


In [24]:
poll_2014_df = poll_df.loc[poll_df['Year'].isin(['2014'])]
poll_2014_df.head()

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
1464412,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.363636,0.204545
1464413,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.363636,0.200000
1464414,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.242857,0.204545
1464415,Alabama,Jefferson,2014-01-01,2014,16.5125,0.013250,0.242857,0.200000
1464416,Alabama,Jefferson,2014-01-02,2014,3.6625,0.018458,-0.009091,0.158824


In [25]:
bad_NO2_2014 = (poll_2014_df[poll_2014_df["NO2 Mean (PPB)"] > 53])
bad_NO2_2014

,State,County,Date,Year,NO2 Mean (PPB),O3 Mean (PPM),SO2 Mean (PPB),CO Mean (PPM)
1464432,Alabama,Jefferson,2014-01-08,2014,58.581818,0.004588,3.525000,0.705556
1464433,Alabama,Jefferson,2014-01-08,2014,58.581818,0.004588,3.525000,0.688235
1464434,Alabama,Jefferson,2014-01-08,2014,58.581818,0.004588,3.350000,0.705556
1464435,Alabama,Jefferson,2014-01-08,2014,58.581818,0.004588,3.350000,0.688235
1488440,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.166667,1.257143
1488441,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.166667,1.233333
1488442,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.166667,1.172714
1488443,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.166667,1.125000
1488444,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.200000,1.257143
1488445,California,Los Angeles,2014-02-14,2014,53.076190,0.010476,1.200000,1.233333


In [26]:
bad_NO2_2014 = (poll_2014_df.loc[poll_2014_df["NO2 Mean (PPB)"] > 53, ["State", "NO2 Mean (PPB)"]]).groupby(["State"]).count()
bad_NO2_2014

,NO2 Mean (PPB)
State,
Alabama,4
California,28
Colorado,32
New Jersey,8


In [27]:
total_baddays_2014 = bad_NO2_2014.join(bad_O3_2014, on="State", how="outer")
total_baddays_2014

,NO2 Mean (PPB),O3 Mean (PPM)
State,,
Alabama,4,NaN
California,28,12.0
Colorado,32,NaN
New Jersey,8,NaN


In [28]:
pollution_data_2014 = total_baddays_2014.rename(columns={"O3 Mean (PPM)":"O3 Mean (PPM) 2014", "NO2 Mean (PPB)":"NO2 Mean (PPB) 2014"})
pollution_data_2014

,NO2 Mean (PPB) 2014,O3 Mean (PPM) 2014
State,,
Alabama,4,NaN
California,28,12.0
Colorado,32,NaN
New Jersey,8,NaN


In [29]:
pollution_data = pd.merge(pollution_data_2010, pollution_data_2014, on="State", how="outer")
pollution_data

,County,Date,Year,NO2 Mean (PPB)_x,O3 Mean (PPM) 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y,NO2 Mean (PPB) 2014,O3 Mean (PPM) 2014
State,,,,,,,,,,
California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0,28.0,12.0
Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN
Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN,NaN,NaN
Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN
Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN
New Jersey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [30]:
final_table = pollution_data.rename(columns={"O3 Mean (PPM) 2010":"O3 Bad Days 2010", "NO2 Mean (PPB) 2010":"NO2 Bad Days 2010","O3 Mean (PPM) 2014":"O3 Bad Days 2014", "NO2 Mean (PPB) 2014":"NO2 Bad Days 2014"})
final_table

,County,Date,Year,NO2 Mean (PPB)_x,O3 Bad Days 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y,NO2 Bad Days 2014,O3 Bad Days 2014
State,,,,,,,,,,
California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0,28.0,12.0
Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN
Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN,NaN,NaN
Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN
Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN
New Jersey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [31]:
final_table.reset_index(level = 0, inplace = True)
final_table

,State,County,Date,Year,NO2 Mean (PPB)_x,O3 Bad Days 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y,NO2 Bad Days 2014,O3 Bad Days 2014
0,California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0,28.0,12.0
1,Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN
2,Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN,NaN,NaN
3,Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN
4,Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
5,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
6,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN
7,New Jersey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [32]:
final_table = final_table.rename(columns={"State":"State Name"})
final_table

,State Name,County,Date,Year,NO2 Mean (PPB)_x,O3 Bad Days 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y,NO2 Bad Days 2014,O3 Bad Days 2014
0,California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0,28.0,12.0
1,Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN
2,Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN,NaN,NaN
3,Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN
4,Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
5,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
6,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN
7,New Jersey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [33]:
x = final_table
x

,State Name,County,Date,Year,NO2 Mean (PPB)_x,O3 Bad Days 2010,SO2 Mean (PPB),CO Mean (PPM),NO2 Mean (PPB)_y,NO2 Bad Days 2014,O3 Bad Days 2014
0,California,4.0,4.0,4.0,4.0,4.0,4.0,4.0,16.0,28.0,12.0
1,Ohio,8.0,8.0,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN
2,Tennessee,24.0,24.0,24.0,24.0,24.0,24.0,24.0,NaN,NaN,NaN
3,Virginia,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN
4,Pennsylvania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
5,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
6,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN
7,New Jersey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN


In [34]:
xfinaltable = (final_table[['State Name', 'O3 Bad Days 2010', 'NO2 Mean (PPB)_y', 'NO2 Bad Days 2014', 'O3 Bad Days 2014']])
xfinaltable

,State Name,O3 Bad Days 2010,NO2 Mean (PPB)_y,NO2 Bad Days 2014,O3 Bad Days 2014
0,California,4.0,16.0,28.0,12.0
1,Ohio,8.0,NaN,NaN,NaN
2,Tennessee,24.0,NaN,NaN,NaN
3,Virginia,4.0,NaN,NaN,NaN
4,Pennsylvania,NaN,12.0,NaN,NaN
5,Alabama,NaN,NaN,4.0,NaN
6,Colorado,NaN,NaN,32.0,NaN
7,New Jersey,NaN,NaN,8.0,NaN


In [35]:
xfinaltable = xfinaltable.rename(columns={"NO2 Mean (PPB)_y":"NO2 Bad Days 2010"})
xfinaltable

,State Name,O3 Bad Days 2010,NO2 Bad Days 2010,NO2 Bad Days 2014,O3 Bad Days 2014
0,California,4.0,16.0,28.0,12.0
1,Ohio,8.0,NaN,NaN,NaN
2,Tennessee,24.0,NaN,NaN,NaN
3,Virginia,4.0,NaN,NaN,NaN
4,Pennsylvania,NaN,12.0,NaN,NaN
5,Alabama,NaN,NaN,4.0,NaN
6,Colorado,NaN,NaN,32.0,NaN
7,New Jersey,NaN,NaN,8.0,NaN


In [36]:
connection_string = "root:MOTigers#21@localhost:3306/pollution_population_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [37]:
xfinaltable.to_sql(name='num_bad_days', con=engine, if_exists='append', index=False)